TODO: Readd a bunch of 'none' types into the newnist_dataset.csv to see if we can get the none accuracy up
^ Didn't work very well, maybe try looking more at the defficiencies of the validation csv

In [1]:
from ml_utils import grab_labels, feature_from_file, translate_bytes, feature_from_list, LabelEncoder, multilabel_to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

#naivetruth_path = "/Users/ryan/Documents/CS/CDAC/xtract_autoencoder/automated_training_results/matio_results/newnist_dataset_raw.csv"
#naivetruth_path = "/Users/ryan/Documents/CS/CDAC/xtract_autoencoder/automated_training_results/matio_results/newnist_dataset_balanced.csv"
naivetruth_path = "/Users/ryan/Documents/CS/CDAC/xtract_autoencoder/automated_training_results/matio_results/newnist_validation.csv"
labels, file_paths = grab_labels(naivetruth_path)

le = LabelEncoder()

y = np.array(multilabel_to_categorical(le.fit_transform(labels)))

x = feature_from_list(file_paths)
x = translate_bytes(x) / 255

x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True)

In [ ]:
print(le.keys)

In [ ]:
print(le.keys)

counts = {}

for label_col in range(7):
    y_true_label = sum(y[:, label_col])
    counts.update({label_col + 1: y_true_label})

for thing in counts:
    print("{}: {} / {}, {}%".format(le.inverse_transform([thing]), counts[thing], len(y), (counts[thing] / len(y)) * 100))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

classifier_model = Sequential()
classifier_model.add(Dense(512, activation='relu', input_shape=(512,)))
classifier_model.add(Dense(256, activation='relu'))
classifier_model.add(Dense(128, activation='relu'))
classifier_model.add(Dense(64, activation='relu'))
classifier_model.add(Dense(32, activation='relu'))
classifier_model.add(Dense(16, activation='relu'))
classifier_model.add(Dense(len(y[0]), activation='sigmoid'))

classifier_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
classifier_model.summary()

history = classifier_model.fit(x_train, y_train,
                               epochs=20,
                               batch_size = 16,
                               shuffle=True,)
                               #validation_data=(x_test, y_test)) 

In [ ]:
from sklearn.metrics import accuracy_score
predict = list(map((lambda x: [1 if i > 0.5 else 0 for i in x]), (classifier_model.predict(x_test, verbose=True))))

print(accuracy_score(y_test, predict))

In [4]:
from keras.models import Sequential
from keras.layers import Conv1D, Dense, MaxPooling1D, GlobalMaxPooling1D, Reshape, Flatten


classifier_model = Sequential()
classifier_model.add(Reshape((len(x[0]), 1), input_shape=(len(x[0]),)))
classifier_model.add(Conv1D(100, 16, activation='relu'))
classifier_model.add(MaxPooling1D(pool_size=2))
classifier_model.add(Conv1D(100, 16, activation='relu'))
classifier_model.add(GlobalMaxPooling1D())
classifier_model.add(Dense(50, activation='relu'))
classifier_model.add(Dense(32, activation='relu'))
classifier_model.add(Dense(28, activation='relu'))
classifier_model.add(Dense(24, activation='relu'))
classifier_model.add(Dense(16, activation='relu'))
classifier_model.add(Dense(8, activation='relu'))
classifier_model.add(Dense(len(y[0]), activation='sigmoid'))

classifier_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier_model.summary()

history = classifier_model.fit(x, y,
                               epochs=3,
                               batch_size = 16,
                               shuffle=True,
                               validation_data=(x_validation, y_validation)) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 512, 1)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 497, 100)          1700      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 248, 100)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 233, 100)          160100    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 32)               

KeyboardInterrupt: 

In [ ]:
import os
import json

model_number = 1
base_dir = "/Users/ryan/Documents/CS/CDAC/xtract_autoencoder/models/matio"

while "matio_classifier_{}.h5".format(str(model_number)) in os.listdir(base_dir):
    model_number += 1

classifier_model.save(os.path.join(base_dir, "matio_classifier_{}.h5".format(str(model_number))))

with open(os.path.join(base_dir, "matio_classifier_{}.json".format(str(model_number))), 'w') as f:
    json_string = classifier_model.to_json()
    json.dump(json_string, f)
#classifier_model.save('matio_classifierV1.h5')

In [ ]:
%%capture --no-display 

import matplotlib.pyplot as plt

# set up figure
f = plt.figure(figsize=(12,6))
f.add_subplot(1,2, 1)

# plot accuracy as a function of epoch
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')

# plot loss as a function of epoch
f.add_subplot(1,2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')
plt.show(block=True)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

predict = list(map((lambda x: [1 if i > 0.5 else 0 for i in x]), (classifier_model.predict(x, verbose=True))))
true = y
validation_accuracy = accuracy_score(true, predict)

print(validation_accuracy)

In [2]:
import keras
import random
from sklearn.metrics import accuracy_score, recall_score, precision_score

#classifier_model = keras.models.load_model('/Users/ryan/Documents/CS/CDAC/xtract_autoencoder/models/matio/matio_classifier_3.h5')
#validation_path = "/Users/ryan/Documents/CS/CDAC/xtract_autoencoder/automated_training_results/matio_results/newnist_validation.csv"
validation_path = "/Users/ryan/Documents/CS/CDAC/xtract_autoencoder/automated_training_results/matio_results/newnist_dataset.csv"

validation_labels, validation_file_paths = grab_labels(validation_path)
# c = list(zip(validation_labels, validation_file_paths))
# random.shuffle(c)
# validation_labels, validation_file_paths = zip(*c)
# validation_labels = list(validation_labels)
# validation_file_paths = list(validation_file_paths)

y_validation = np.array(multilabel_to_categorical(le.transform(validation_labels)))

x_validation = feature_from_list(validation_file_paths)
x_validation = translate_bytes(x_validation) / 255

Using TensorFlow backend.


In [ ]:
#print(classifier_model.predict(x_validation, verbose=True))
validation_predict = list(map((lambda x: [1 if i > 0.5 else 0 for i in x]), (classifier_model.predict(x_validation, verbose=True))))
#validation_predict = classifier_model.predict(x_validation, verbose=True)

print(accuracy_score(y_validation, validation_predict))

In [ ]:
from keras.metrics import accuracy

#print(accuracy(y_validation, validation_predict))
print(sum(accuracy(y_validation, validation_predict)) / len(accuracy(y_validation, validation_predict)))

In [ ]:
from ml_utils import convert_to_index, plot_confusion_matrix
from pycm import ConfusionMatrix
import numpy as np

y_true = np.array(y_validation)
y_pred = np.array(predict)

labels = list(le.keys.keys())

conf_mat_dict={}


for label_col in range(len(labels)):
    y_true_label = y_true[:, label_col]
    y_pred_label = y_pred[:, label_col]
    conf_mat_dict[labels[label_col]] = ConfusionMatrix(y_true_label, y_pred_label)


for label, matrix in conf_mat_dict.items():
    print("label: {}".format(label))
    plt.figure()
    plot_confusion_matrix(matrix, title='confusion matrix')